# Set-up

## Installing libraries and libcudnn8

In [1]:
## only with GPU
FILEID = "1-bPsREsUCOiJHzIqi8DQrfSjTAf5VAW_"
!apt-get install --allow-change-held-packages libcudnn8=8.1.1.33-1+cuda11.2 -y
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.EEG_Tensorflow_models.git
!pip install mne
!pip install scikeras[tensorflow]
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
!unzip MI_EEG_ClassMeth.zip #Package with useful functions for motor imagery classification based in EEG.
!dir




The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 91 not upgraded.
Need to get 421 MB of archives.
After this operation, 2621 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  libcudnn8 8.1.1.33-1+cuda11.2 [421 MB]
Fetched 421 MB in 5s (85.2 MB/s)
(Reading database ... 108827 files and directories currently installed.)
Removing libcudnn8-dev (8.0.5.39-1+cuda11.0) ...
update-alternatives: removing manually selected alternative - switching libcudnn to auto mode
(Reading database ... 108805 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.1.1.33-1+cuda11.2_amd64.deb ...
Unpacking libcudnn8 (8.1.1.33-1+cuda11.2) over (8.0.5.39-1+cuda11.0) ...
Setting up libcudnn8 (8.1.1.33-1+cuda11.2) ...
  Cloning https://github.com/UN-GCPDS/python-gcp

## Import libraries

In [2]:
# gcpds datasets
from gcpds.databases import GIGA_MI_ME
from gcpds.databases.BCI_Competition_IV import Dataset_2a
import gcpds.databases as loaddb

# freq filter 
from MI_EEG_ClassMeth.FeatExtraction import TimeFrequencyRpr

# general
import numpy as np
from scipy.signal import resample
import pickle
import warnings
import mne
from time import time
warnings.filterwarnings('ignore')

# tensorlfow 
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D, AveragePooling2D, BatchNormalization, Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Layer
from tensorflow.keras.regularizers import L1L2

# scikeras
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV,StratifiedShuffleSplit
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, cohen_kappa_score, roc_auc_score

2022-11-22 01:47:41.667778: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-22 01:47:42.862773: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.7/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-11-22 01:47:42.863032: W tensorflow/compiler/xl

## Define functions

In [3]:
def kappa(y_true, y_pred):
    return cohen_kappa_score(np.argmax(y_true, axis = 1),np.argmax(y_pred, axis = 1))

## GIGA Science dataset

In [4]:
def load_GIGA(db: GIGA_MI_ME,
              sbj: int,
              fs: float, 
              f_bank: np.ndarray, 
              vwt: np.ndarray, 
              new_fs: float) -> np.ndarray:

    eeg_ch_names = ['Fp1','Fpz','Fp2',
              'AF7','AF3','AFz','AF4','AF8',
              'F7','F5','F3','F1','Fz','F2','F4','F6','F8',
              'FT7','FC5','FC3','FC1','FCz','FC2','FC4','FC6','FT8',
              'T7','C5','C3','C1','Cz','C2','C4','C6','T8',
              'TP7','CP5','CP3','CP1','CPz','CP2','CP4','CP6','TP8',
              'P9','P7','P5','P3','P1','Pz','P2','P4','P6','P8','P10',
              'PO7','PO3','POz','PO4','PO8',
              'O1','Oz','O2',
              'Iz']

    index_eeg_chs = db.format_channels_selectors(channels = eeg_ch_names) - 1

    tf_repr = TimeFrequencyRpr(sfreq = fs, f_bank = f_bank, vwt = vwt)

    db.load_subject(sbj)
    X, y = db.get_data(classes = ['left hand mi', 'right hand mi']) #Load MI classes, all channels {EEG}, reject bad trials, uV
    X = X[:,index_eeg_chs,:] #spatial rearrangement
    X = np.squeeze(tf_repr.transform(X))
    #Resampling
    if new_fs == fs:
        print('No resampling, since new sampling rate same.')
    else:
        print("Resampling from {:f} to {:f} Hz.".format(fs, new_fs))
        X = resample(X, int((X.shape[-1]/fs)*new_fs), axis = -1)
    return X, y

## BCI2a dataset

In [5]:
def load_BCICIV2a(db: Dataset_2a,
               sbj: int,
               fs: float, 
               f_bank: np.ndarray, 
               vwt: np.ndarray, 
               new_fs: float) -> np.ndarray:

    tf_repr = TimeFrequencyRpr(sfreq = fs, f_bank = f_bank, vwt = vwt)
    # training 
    db.load_subject(sbj, mode = 'training')
    X_tr, y_tr = db.get_data() #Load all classes, all channels {EEG, EOG}, reject bad trials
    X_tr = X_tr[:,:-3,:] # pick EEG channels
    X_tr = X_tr*1e6 #uV
    X_tr = np.squeeze(tf_repr.transform(X_tr))
    # testing
    db.load_subject(sbj, mode = 'evaluation')
    X_ts, y_ts = db.get_data() #Load all classes, all channels {EEG, EOG}, reject bad trials
    X_ts = X_ts[:,:-3,:] # pick EEG channels
    X_ts = X_ts*1e6 #uV
    X_ts = np.squeeze(tf_repr.transform(X_ts))
    # merge both training and evaluation sets  
    X = np.concatenate([X_tr, X_ts], axis = 0)
    y = np.concatenate([y_tr, y_ts], axis = 0)    
    # Resampling
    if new_fs == fs:
        print('No resampling, since new sampling rate same.')
    else:
        print("Resampling from {:f} to {:f} Hz.".format(fs, new_fs))
        X = resample(X, int((X.shape[-1]/fs)*new_fs), axis = -1)
    return X, y

## Define the model EEGnet

In [6]:
from EEG_Tensorflow_models.Models import DeepConvNet, ShallowConvNet, EEGNet

# Experiment

## Experiment configuration 

In [7]:
seed=23
folds=5
epochs_train = 500
data_set='GIGA_MI_ME'## BCICIV2a # GIGA_MI_ME

save_folder = 'Shallownet_128Hz'
Experiment=f'{data_set}__{folds}_fold'

model_name= f'{save_folder}{epochs_train}_epoch'


In [8]:
PATH=f'./{save_folder}/'
!mkdir '{PATH}'

## Run experiment

In [9]:
tf.random.set_seed(seed)

if data_set=='GIGA_MI_ME':
    subjects = np.arange(52)+1
    subjects = np.delete(subjects,[28,33])
    db = GIGA_MI_ME('../input/giga-mi-me/GIGA_MI_ME/') # update path
    num_class = 2
    load_args = dict(db = db,
                fs = db.metadata['sampling_rate'],
                f_bank = np.asarray([[4., 40.]]),
                vwt = np.asarray([[2.5, 5]]),
                new_fs = 128.0)    
elif data_set=='BCICIV2a':
    subjects = np.arange(9)+1
    db = Dataset_2a('../input/bciciv2a/BCI_CIV_2a/') # update path
    num_class = 4
    load_args = dict(db = db,
                fs = db.metadata['sampling_rate'],
                f_bank = np.asarray([[4., 40.]]),
                vwt = np.asarray([[2.5, 6]]),
                new_fs = 128.0)
else:
    print('Select a valid dataset')            

t=time()
for sbj in subjects[25:]:
    load_args['sbj'] = sbj

    if data_set=='GIGA_MI_ME':
        X_train, y_train = load_GIGA(**load_args)
    elif data_set=='BCICIV2a':
        X_train, y_train = load_BCICIV2a(**load_args)

    X_train = X_train[..., np.newaxis]
    Y_train = tf.keras.utils.to_categorical(y_train,num_classes=num_class)
    # build model
    clf = KerasClassifier(
        DeepConvNet,
        random_state=seed,

        #model hyperparameters
        nb_classes=num_class, 
        Chans = X_train.shape[1], 
        Samples = X_train.shape[2],
        version='2018',

        #model config
        verbose=0,
        batch_size=500, #full batch        
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer="adam",
        optimizer_learning__rate=0.1,
        metrics = ['accuracy'],
        epochs = epochs_train
    )
    # search params
    param_grid =  {
                'version':['2018'],
                }
    
    #Gridsearch
    scoring = {"AUC": 'roc_auc', "Accuracy": make_scorer(accuracy_score),'Kappa':make_scorer(kappa)}
    
    cv = GridSearchCV(clf,param_grid,cv=StratifiedShuffleSplit(n_splits = folds, test_size = 0.2, random_state = seed),
                         verbose=0,n_jobs=1,
                         scoring=scoring,
                         refit="Accuracy",
                            )
    # frind best params with gridsearch
    cv.fit(X_train,Y_train)
    # best score
    print('Subject',sbj,'Accuracy',cv.best_score_,'elapsed time',time()-t)
    print('---------')
    
    cv.cv_results_['best_index_']=cv.best_index_
    
    #########
    cv.best_estimator_.model_.save_weights(f'{PATH}Model_{Experiment}_{model_name}_sujeto_{sbj}_'+data_set+'_4_40_weights.h5')
    with open(PATH+'Results_'+Experiment+'_'+model_name+'_sujeto_'+str(sbj)+'_'+data_set+'_4_40.p','wb') as f:
        pickle.dump(cv.cv_results_,f)     

Resampling from 512.000000 to 128.000000 Hz.


2022-11-22 01:47:59.078275: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/Dp_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2022-11-22 01:48:00.948103: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-11-22 01:48:02.386370: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022

Subject 26 Accuracy 0.77 elapsed time 729.5166027545929
---------
Resampling from 512.000000 to 128.000000 Hz.


2022-11-22 02:00:06.946167: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_6/Dp_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2022-11-22 02:01:48.653293: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_7/Dp_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2022-11-22 02:04:12.894158: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_8/Dp_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2022-11-22 02:06:37.087783: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_9/Dp_1/dropout/SelectV

Subject 27 Accuracy 0.43243243243243246 elapsed time 1558.9840264320374
---------
Resampling from 512.000000 to 128.000000 Hz.


...

Subject 50 Accuracy 0.6449999999999999 elapsed time 17624.351778268814
---------
Resampling from 512.000000 to 128.000000 Hz.


2022-11-22 06:41:41.060919: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_138/Dp_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2022-11-22 06:44:06.357283: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_139/Dp_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2022-11-22 06:45:52.885689: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_140/Dp_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2022-11-22 06:47:39.264146: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_141/Dp_1/dropout

Subject 51 Accuracy 0.5487179487179488 elapsed time 18415.840117692947
---------
Resampling from 512.000000 to 128.000000 Hz.


2022-11-22 06:54:53.400517: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_144/Dp_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2022-11-22 06:57:17.769749: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_145/Dp_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2022-11-22 06:59:06.338968: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_146/Dp_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2022-11-22 07:01:31.393401: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_147/Dp_1/dropout

Subject 52 Accuracy 0.525 elapsed time 19182.25148677826
---------


In [10]:
!rm -r ./MI_EEG_ClassMeth
!rm -r ./__MACOSX
!rm ./MI_EEG_ClassMeth.zip